# Implementing LiDAR-NeRF with Ray Drop

In [ ]:
#load requirements for working with PCs
from vedo import *
from ipyvtklink.viewer import ViewInteractiveWidget
import numpy as np
import tensorflow as tf
import time
import cv2

#limit GPU memory ------------------------------------------------
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  try:
    memlim = 20*1024
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memlim)])
  except RuntimeError as e:
    print(e)
#-----------------------------------------------------------------

import sys
import os
current = os.getcwd()
parent_directory = os.path.dirname(current)
sys.path.append(parent_directory)
sys.path.append(parent_directory+"/point_cloud_rectification")
from ICET_spherical import ICET
from linear_corrector import LC

from utils import R_tf
from metpy.calc import lat_lon_grid_deltas
from scipy.spatial.transform import Rotation as R
from matplotlib import pyplot as plt
import copy
import trimesh


from pillow_heif import register_heif_opener
from matplotlib import pyplot as p
from colmapParsingUtils import *
from scipy.spatial.transform import Rotation as R
import cv2

from tqdm import tqdm_notebook as tqdm
from PIL import Image

tf.compat.v1.enable_eager_execution()


%load_ext autoreload
%autoreload 2
%autosave 180
# %matplotlib notebook

## Load Gazebo Point Clouds and Convert to image-like data format

In [ ]:
n_images =  17 #18 max for town, 11 max for old box, 16 for new box
n_rots = 8 #8 #1 #number of side by side frames to add in
# n_rots = 260 #130 for 1/16th #60 for 1/8th (needs to be much bigger when we are splitting up range images vertically)

#Get LiDAR Intrinsics
#Simulated LIDAR Sensor -- (look for <min_angle> in velodyne.world file in ROS package)
phimin = -0.53529248 #rad
phimax = 0.18622663 #rad
#achieve roughly same horizonal and vertical fov
vert_fov = np.rad2deg(phimax-phimin)
# images = np.zeros([n_images, 64, image_width, 1])
# images = np.zeros([n_images*n_rots, 64, 64, 1]) #squares
# images = np.zeros([n_images*n_rots, 64, 2, 1]) #2 pixels wide
poses = np.zeros([n_images*n_rots,4,4])
# non_returns = np.zeros([n_images*n_rots, 64, 64, 1]) #squares
images = np.ones([n_images*n_rots, 64, 64, 2]) #depth image and raydrop

#focal length (in pixels) = Image Size / (2 tan(FOV/2)) #needs to be array!
focal = np.array(np.shape(images)[1]/(2*np.tan((phimax-phimin)/2))) #default image size
H, W = images.shape[1:3]
print(focal, H, W)

gtfn = "gazebo_scene/ground_truth.npy" #town
# gtfn = "gazebo_scene2/ground_truth.npy" #box v1
# gtfn = "gazebo_scene2/box_ground_truth.npy" #box v2
# gtfn = "gazebo_scene2/box2_ground_truth.npy" #box v3
sensor_pose = np.load(gtfn)
# print(sensor_pose)
print(np.shape(sensor_pose))

for i in range(n_images):
#     for j in range(n_rots):
    #load point cloud file
    pcfn = "gazebo_scene/scan" + str(i+2) + ".npy" #town
#     pcfn = "gazebo_scene2/scan" + str(i+2) + ".npy" #box v1
#         pcfn = "gazebo_scene2/box_scan" + str(i+2) + ".npy" #box v2
#     pcfn = "gazebo_scene2/box2_scan" + str(i+2) + ".npy" #box v3
    pc = np.load(pcfn)
    
    #cap inf values TEMP
    pc[pc[:,0]>64] = 100 #64 #0
    pc[pc[:,0]<-64] = 100 #64 #0
    
    #convert to image array
    pc_spherical = LC.c2s(LC,pc).numpy() #[r, theta, phi]

    for j in range(n_rots):

        pcs = np.reshape(pc_spherical, [-1,64,3])
        pcs = np.flip(pcs, axis = 1) #flip vertical
        pcs = np.flip(pcs, axis = 0) #flip horizontal to look at first clockwise patch of scan sweep 

        #resize image to 64x64
        image_width = int((vert_fov/360)*np.shape(pcs)[0])
        
        pcs = pcs[j*image_width:(j+1)*image_width,:,0].T ##square
#         pcs = pcs[(j*image_width//32):((j+1)*image_width//32),:,0].T #32nds

        pcs = cv2.resize(pcs, (64, 64), cv2.INTER_NEAREST) #keep square
        #preserve aspect ratio and focal length but just take middle
#         pcs = pcs[:,31:33] #uncomment for 32nds
        images[j+(i*n_rots),:,:,0] = pcs #save depth information to first channel
    

        a = np.argwhere(pcs > 64)
#         non_returns[j+(i*n_rots),a[:,0],a[:,1],0] = 1
        images[j+(i*n_rots),a[:,0],a[:,1],1] = 0 #save raydrop mask to 2nd channel
    
        #get sensor transformation matrix
        rotm = np.eye(4)
        rotm[0,3] = sensor_pose[i+1,0] #x
        rotm[2,3] = -sensor_pose[i+1,1] #y
        rotm[1,3] = -sensor_pose[i+1,2] #z
        rotm[:3,:3] = R.from_quat(sensor_pose[i+1,3:]).as_matrix() 
        
        #account for image crop in rotation
        crop_angle = -(phimax-phimin)/2 - j*(phimax-phimin) #square
#         crop_angle = -(phimax-phimin)/64 - j*(phimax-phimin)/32 #2-pixels wide
        rotm_crop = R.from_euler('xyz', [0,-crop_angle,0]).as_matrix()
        rotm[:3,:3] = rotm[:3,:3] @ rotm_crop

        #also need to account for the fact that the LIDAR beam isn't actually centered at horizon
        sensor_elevation_zero_rotm = R.from_euler('xyz', [(phimin+phimax)/2,0,0]).as_matrix() #correct
        rotm[:3,:3] = rotm[:3,:3] @ sensor_elevation_zero_rotm

        #flip x and z axis
        rotm[0,-1], rotm[2,-1] = rotm[2,-1], rotm[0,-1] 

        # flip sign of y and z axis
        rotm[0:3,2] *= -1 
        rotm[0:3,1] *= -1
        rotm = rotm[[1,0,2,3],:]
        rotm[2,:] *= -1 # flip whole world upside down
#         #translate all frames above xy plane
        rotm[2,-1] += 25 #for town  
#         rotm[2,-1] += 5 #for box (old)
#         rotm[2,-1] += 7 #for box (new)

# #         scale translations
#         rotm[:3,-1] *= 0.1
#         #scale ranges measurements?
#         images[j+(i*n_rots),:,:,0] *= 0.1
        
        poses[j+(i*n_rots)] = rotm

fig, ax = p.subplots(1,4)
ax[0].imshow(images[-4,:,:,0], cmap = "gray", norm='log')
ax[1].imshow(images[-3,:,:,0], cmap = "gray", norm='log')
ax[2].imshow(images[-2,:,:,0], cmap = "gray", norm='log')
ax[3].imshow(images[-1,:,:,0], cmap = "gray", norm='log')
images = images.astype(np.float32)
poses = poses.astype(np.float32)

# test on one only
testimg, testpose = images[(n_images*n_rots)-1], poses[(n_images*n_rots)-1]
images = images[:((n_images*n_rots)-1),...,:3]
poses = poses[:((n_images*n_rots)-1)]

# #90/10 split
# cutoff = (n_images*9)//10
# print(cutoff)
# testimg, testpose = images[cutoff:], poses[cutoff:]
# images = images[:cutoff,...,:3]
# poses = poses[:cutoff]

##  Train Vanilla NeRF on depth image data

In [ ]:
def posenc(x):
  rets = [x]
  for i in range(L_embed):
    for fn in [tf.sin, tf.cos]:
      rets.append(fn(2.**i * x))
  return tf.concat(rets, -1)

L_embed = 6 #6
embed_fn = posenc
# L_embed = 0
# embed_fn = tf.identity

def init_model(D=8, W=256): #8,256
    relu = tf.keras.layers.ReLU()    
    dense = lambda W=W, act=relu : tf.keras.layers.Dense(W, activation=act)

    inputs = tf.keras.Input(shape=(3 + 3*2*L_embed)) 
    outputs = inputs
    for i in range(D):
        outputs = dense()(outputs)
        if i%4==0 and i>0:
            outputs = tf.concat([outputs, inputs], -1)
            
    #umcomment below for non-raydrop model
#     outputs = dense(1, act=None)(outputs) #[depth] output only
##     outputs = dense(2, act=None)(outputs) #[depth, ray drop] #does not work
#     model = tf.keras.Model(inputs=inputs, outputs=outputs)

    #extend "color" channels to small MLP after output of density channel
    sigma_channel = dense(1, act=None)(outputs)
    
    #start ray drop branch
    rd_start = tf.concat([outputs, inputs], -1)
    rd_channel = dense(128, act=relu)(outputs)
#     rd_channel = dense(128, act=relu)(rd_channel)
    rd_channel = dense(128, act=relu)(rd_channel)
    rd_channel = dense(1, act=tf.keras.activations.sigmoid)(rd_channel)
    out = tf.concat([sigma_channel, rd_channel], -1)
    model = tf.keras.Model(inputs=inputs, outputs=out)
    
    return model


def get_rays(H, W, focal, c2w):
    i, j = tf.meshgrid(tf.range(W, dtype=tf.float32), tf.range(H, dtype=tf.float32), indexing='xy')
#     #pinhold camera projection model (TinyNeRF)~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#     dirs = tf.stack([(i-W*.5)/focal, -(j-H*.5)/focal, -tf.ones_like(i)], -1)
#     rays_d = tf.reduce_sum(dirs[..., np.newaxis, :] * c2w[:3,:3], -1)

    #spherical projection model (ours)~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    phimin = -0.53529248 #rad
    phimax = 0.18622663 #rad

    dirs_test = tf.stack([-tf.ones_like(i), 
                          -(j-(W*.5))/(focal) - (np.pi/2) - phimax, 
                          np.arctan((i-(H*.5))/(focal))], -1) #square
#                           np.arctan((i-(H*.5))/(focal*32))], -1) #2 pixels wide
    
    dirs_test = tf.reshape(dirs_test,[-1,3])
    #[r, theta, phi] --> [r, phi, theta]
    dirs_test = LC.s2c(LC, tf.transpose(tf.Variable([dirs_test[:,0], dirs_test[:,2], dirs_test[:,1]], dtype = tf.float32)))
    
    rotm = R.from_euler('xyz', [0,-(np.pi/2)+(((phimax+phimin))/2),0]).as_matrix()
    dirs_test = dirs_test @ rotm
    dirs_test = dirs_test @ tf.transpose(c2w[:3,:3])
    dirs = dirs_test @ (c2w[:3,:3] 
                          @ R.from_euler('xyz', [0,0,np.pi/2]).as_matrix()
                          @ np.linalg.pinv(c2w[:3,:3]) )

    dirs = tf.reshape(dirs, [64,64,3]) #square
#     dirs = tf.reshape(dirs, [64,2,3]) #2 pixels wide
    
    rays_d = tf.reduce_sum(dirs[..., np.newaxis, :] * np.eye(3), -1) 
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~    
        
    rays_o = tf.broadcast_to(c2w[:3,-1], tf.shape(rays_d))
    return rays_o, rays_d


def render_rays(network_fn, rays_o, rays_d, near, far, N_samples, rand=True):

    def batchify(fn, chunk=1024*512): #1024*128 converged for box3 #1024*32 in TinyNeRF
        return lambda inputs : tf.concat([fn(inputs[i:i+chunk]) for i in range(0, inputs.shape[0], chunk)], 0)
    
    # Compute 3D query points
    z_vals = tf.linspace(near, far, N_samples) 
    if rand:
      z_vals += tf.random.uniform(list(rays_o.shape[:-1]) + [N_samples]) * (far-near)/N_samples
    #[image_height, image_width, batch_size, 3]
    pts = rays_o[...,None,:] + rays_d[...,None,:] * z_vals[...,:,None]
    
    # Run network
    pts_flat = tf.reshape(pts, [-1,3])
    pts_flat = embed_fn(pts_flat)
    raw = batchify(network_fn)(pts_flat)
#     raw = tf.reshape(raw, list(pts.shape[:-1]) + [4]) #OG nerf
#     raw = tf.reshape(raw, list(pts.shape[:-1]) + [1])  #[depth]
    raw = tf.reshape(raw, list(pts.shape[:-1]) + [2]) # [depth, ray drop] 
    
    # Compute opacities and colors
    #OG TinyNeRF
#     sigma_a = tf.nn.relu(raw[...,3])
#     rgb = tf.math.sigmoid(raw[...,:3]) 
    #LiDAR-NeRF
    sigma_a = tf.nn.relu(raw[...,0])
    ray_drop = tf.nn.relu(raw[...,1])
#     ray_drop = tf.math.sigmoid(raw[...,1]) #test

    # Do volume rendering
    dists = tf.concat([z_vals[..., 1:] - z_vals[..., :-1], tf.broadcast_to([1e10], z_vals[...,:1].shape)], -1) 
    alpha = 1.-tf.exp(-sigma_a * dists)  
    weights = alpha * tf.math.cumprod(1.-alpha + 1e-10, -1, exclusive=True)

#     print("weights[...,None]",np.shape(weights[..., None]))
#     print("ray_drop", np.shape(ray_drop))
#     print(np.shape(weights), np.shape(z_vals))
    
    depth_map = tf.reduce_sum(weights * z_vals, -1)
    ray_drop_map = tf.reduce_sum(weights * ray_drop, -1) #axis was -2, changed to -1 
    acc_map = tf.reduce_sum(weights, -1)

#     return depth_map, acc_map
    return depth_map, acc_map, ray_drop_map

In [ ]:
model = init_model()
optimizer = tf.keras.optimizers.Adam(5e-4) #default tiny-NeRF
# optimizer = tf.keras.optimizers.Adam(5e-5)

N_samples = 256 #256 #64 #decrease as needed for VRAM
N_iters = 50_000
psnrs = []
iternums = []
i_plot = 50

import time
t = time.time()
for i in range(N_iters+1):
    img_i = np.random.randint(images.shape[0])
    target = images[img_i,:,:,:1]
    target_drop_mask = images[img_i,:,:,1:]
    pose = poses[img_i]
    #get ray origins and ray directions
    rays_o, rays_d = get_rays(H, W, focal, pose)
    
    with tf.GradientTape() as tape:
        #just calculate loss via comparing depth output vs target (should also be depth!)
#         depth, acc = render_rays(model, rays_o, rays_d, near=1., far=64., N_samples=N_samples, rand=True)
        depth, acc, ray_drop = render_rays(model, rays_o, rays_d, near=1., far=64., N_samples=N_samples, rand=True)
        depth = depth[:,:,None]
        ray_drop = ray_drop[:,:,None]
        
#         print(np.shape(depth), np.shape(target))
#         print(np.shape(ray_drop), np.shape(target_drop_mask))
        
        #default loss from TinyNERF
#         loss = tf.reduce_mean(tf.square(depth - target)) 
        #Distance Loss only
#         loss = tf.reduce_mean(tf.abs(depth - target)) # <-- works way better than dist^2
    
        #LiDAR-NeRF LoSS ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        ## L_total = L_dist + lam1*L_intensity + lam2*L_raydrop + lam3*L_reg
        ##          lam1=1, lam2=1, lam3=100
        ## L_reg --> structural regularization: consider gradient loss only on high-texture areas
        
        #Gradient Loss (structural regularization for smooth surfaces)
        thresh = 0.025 #was 0.1 in LiDAR-NeRF
        mask = np.zeros(np.shape(target[:,:,0]))
        vertical_grad_target = np.gradient(target[:,:,0])[0] 
        vertical_past_thresh = np.argwhere(tf.abs(vertical_grad_target) > thresh)
        mask[vertical_past_thresh[:,0], vertical_past_thresh[:,1]] = 1
        horizontal_grad_target = np.gradient(target[:,:,0])[1]
        horizontal_past_thresh = np.argwhere(tf.abs(horizontal_grad_target) > thresh)
        mask[horizontal_past_thresh[:,0], horizontal_past_thresh[:,1]] = 1
        vertical_grad_inference = np.gradient(depth[:,:,0])[0]
        horizontal_grad_inference = np.gradient(depth[:,:,0])[1]
        mag_difference = tf.math.sqrt((vertical_grad_target-vertical_grad_inference)**2 + (horizontal_grad_target-horizontal_grad_inference)**2)
#         L_reg = tf.reduce_mean(np.multiply(mag_difference, mask))
        #suppress ray drop areas
        L_reg = np.multiply(mag_difference, mask)
        L_reg = tf.reduce_mean(tf.multiply(L_reg, target_drop_mask))
        L_reg = tf.cast(L_reg, tf.float32)            
                
        #ray drop loss
        L_raydrop = tf.keras.losses.binary_crossentropy(target_drop_mask, ray_drop)
        L_raydrop = tf.math.reduce_mean(tf.abs(L_raydrop))
#         print("Ray Drop Loss:", L_raydrop)
    
        #distance loss
#         L_dist = tf.reduce_mean(tf.abs(depth - target)) #simple loss
        #suppressing ray drop areas
        depth_nondrop = tf.math.multiply(depth, target_drop_mask)
        target_nondrop = tf.math.multiply(target, target_drop_mask)
        L_dist = tf.reduce_mean(tf.abs(depth_nondrop - target_nondrop))
    
        lam1 = 100
        lam2 = 1 #1/(64**2)
        loss = L_dist + lam1*L_reg + lam2*L_raydrop       

#         ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    if i%i_plot==0:
        t = time.time()
        # Render the holdout view for logging
        rays_o, rays_d = get_rays(H, W, focal, testpose) #constant validation image
#         sampl = int(np.random.uniform(low=0, high=len(testpose)-1))        
#         rays_o, rays_d = get_rays(H, W, focal, testpose[sampl]) #90/10 split
#         depth, acc = render_rays(model, rays_o, rays_d, near=1., far=64., N_samples=N_samples)
        depth, acc, ray_drop = render_rays(model, rays_o, rays_d, near=1., far=64., N_samples=N_samples)
        depth = depth[:,:,None]
        ray_drop = ray_drop[:,:,None]
#         print(ray_drop)
#         #simple depth only
#         loss = tf.reduce_mean(tf.square(depth[:,:,None] - testimg[:,:,0])) 
#         #ray drop only
#         L_raydrop = tf.keras.losses.binary_crossentropy(testimg[:,:,1], ray_drop)
#         loss = tf.math.reduce_sum(tf.abs(L_raydrop)).numpy()

        #LiDAR-NeRF LoSS ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        ## L_total = L_dist + lam1*L_intensity + lam2*L_raydrop + lam3*L_reg
        ##          lam1=1, lam2=1, lam3=100
        ## L_reg --> structural regularization: consider gradient loss only on high-texture areas
        
        target = testimg[:,:,:1]
        target_drop_mask = testimg[:,:,1:]
        
        #Gradient Loss (structural regularization for smooth surfaces)
        thresh = 0.025 #was 0.1 in LiDAR-NeRF
        mask = np.zeros(np.shape(target[:,:,0]))
        vertical_grad_target = np.gradient(target[:,:,0])[0] 
        vertical_past_thresh = np.argwhere(tf.abs(vertical_grad_target) > thresh)
        mask[vertical_past_thresh[:,0], vertical_past_thresh[:,1]] = 1
        horizontal_grad_target = np.gradient(target[:,:,0])[1]
        horizontal_past_thresh = np.argwhere(tf.abs(horizontal_grad_target) > thresh)
        mask[horizontal_past_thresh[:,0], horizontal_past_thresh[:,1]] = 1
        vertical_grad_inference = np.gradient(depth[:,:,0])[0]
        horizontal_grad_inference = np.gradient(depth[:,:,0])[1]
        mag_difference = tf.math.sqrt((vertical_grad_target-vertical_grad_inference)**2 + (horizontal_grad_target-horizontal_grad_inference)**2)
        L_reg = tf.reduce_mean(np.multiply(mag_difference, mask))
#         L_reg = tf.reduce_mean(tf.abs(np.multiply(vertical_grad_target-vertical_grad_inference, mask)))
        L_reg = tf.cast(L_reg, tf.float32)     
        #suppress ray drop areas
        L_reg = np.multiply(mag_difference, mask)
        L_reg = tf.reduce_mean(tf.multiply(L_reg, target_drop_mask))
        L_reg = tf.cast(L_reg, tf.float32)            
    
        #distance loss
        L_dist = tf.reduce_mean(tf.abs(depth - target))
        
        #ray drop loss
        L_raydrop = tf.keras.losses.binary_crossentropy(target_drop_mask, ray_drop)
        L_raydrop = tf.math.reduce_sum(tf.abs(L_raydrop))
#         print("Ray Drop Loss:", L_raydrop)
        
        lam1 = 100
        lam2 = 1/(64**2)
        loss = L_dist + lam1*L_reg + lam2*L_raydrop       

#         ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        psnr = -10. * tf.math.log(loss) / tf.math.log(10.)
        psnrs.append(psnr.numpy())
        iternums.append(i)
        p.figure(figsize=(10,4))
        p.subplot(131)
        p.imshow(depth,cmap = "gray", norm='log')
        p.title(f'Estimated Depth at Iteration: {i}')
#         p.imshow(testimg[:,:,1],cmap = "gray") #, norm='log')
#         p.title(f'Actual Mask at Iteration: {i}')
        p.subplot(133)
        p.plot(iternums, psnrs)
        p.title('PSNR')
        #look at depth map
        p.subplot(132)
        p.imshow(ray_drop, cmap="gray", norm = 'log')
        p.title("estimated ray drop mask")
        p.show()
print('Done')

# Debug: Show Depth Mask fit to Training Poses

In [ ]:
N_samples = 128 #64 #does not have to match what was used in training?? 
idx = 40
# %matplotlib notebook

rotm = poses[idx]
# rotm = testpose

#call NeRF using specified novel rotm
rays_o, rays_d = get_rays(H, W, focal, rotm)
depth, acc, ray_drop = render_rays(model, rays_o, rays_d, near=1., far=64., N_samples=N_samples)
new_point_cloud_spherical = np.zeros([np.shape(depth)[0]*np.shape(depth)[1],3])

depth = tf.transpose(depth).numpy() #need this
depth = np.flip(depth, axis = 0) #needed

fig, ax = p.subplots(1,3)
ax[0].imshow(images[idx,:,:,0],cmap = "gray",norm='log')
ax[0].set_title("depth image")
ax[1].imshow(images[idx,:,:,1],cmap = "gray")
# ax[1].imshow(testimg[:,:,1],cmap = "gray")
ax[1].set_title("true ray drop mask")
ax[2].set_title("estimated ray drop mask")
ax[2].imshow(ray_drop, cmap = "gray")

L_raydrop = tf.keras.losses.binary_crossentropy(images[idx,:,:,1], ray_drop)
# L_raydrop = tf.keras.losses.binary_crossentropy(testimg[:,:,1], ray_drop)
loss = tf.math.reduce_sum(tf.abs(L_raydrop)).numpy()
# print(loss)
# print(L_raydrop)



In [ ]:
# # try suppressing distance loss in  training data with non-returns
# L_dist = tf.reduce_mean(tf.multiply(tf.abs(depth - target), target_drop_mask))
# print(L_dist)

## Draw Full Point Cloud From Novel Frame

In [ ]:
num_rots =  9 #279 
N_samples = 256 #64 #does not have to match what was used in training?? 

plt = Plotter(N = 1, axes = 0, bg = (1, 1, 1), interactive = True) #axes = 4 (simple), 1(scale)
disp=[]

for j in range(num_rots):
# for j in np.linspace(0,7,15):
    #get sensor transformation matrix
    rotm = np.eye(4)

    # account for image crop in rotation -------------------
    crop_angle = -(phimax-phimin)/2 - j*(phimax-phimin)#square
#     crop_angle = -(phimax-phimin)/16 - j*(phimax-phimin)/8 #eighth
#     crop_angle = -(phimax-phimin)/32 - j*(phimax-phimin)/16 #1/16
#     crop_angle = -(phimax-phimin)/32 - j*(phimax-phimin)/32 #1/32
    rotm_crop = R.from_euler('xyz', [0,-crop_angle,0]).as_matrix()
    rotm[:3,:3] = rotm[:3,:3] @ rotm_crop

    #also need to account for the fact that the LIDAR beam isn't actually centered at horizon
    sensor_elevation_zero_rotm = R.from_euler('xyz', [(phimin+phimax)/2,0,0]).as_matrix()
    rotm[:3,:3] = rotm[:3,:3] @ sensor_elevation_zero_rotm
    # ------------------------------------------------------

    # flip x and z axis
    rotm[0,-1], rotm[2,-1] = rotm[2,-1], rotm[0,-1] 

    rotm[0:3,2] *= -1 # flip sign of y and z axis
    rotm[0:3,1] *= -1
    rotm = rotm[[1,0,2,3],:]
    rotm[2,:] *= -1 # flip whole world upside down
    rotm[2,-1] += 30 #4 #x in world frame output
#     rotm[2,-1] += 15 #- i/6 #x in world frame output
    rotm[0,-1] = -1.5 #- (i/3) #z in world frame output
    rotm[1,-1] = 2 #+ (i/6) #y in world frame

    rotm = rotm.astype(np.float32)
    
    start = time.time()
    
    #call NeRF using specified novel rotm
    rays_o, rays_d = get_rays(H, W, focal, rotm)
#     depth, acc = render_rays(model, rays_o, rays_d, near=1., far=64., N_samples=N_samples)
    depth, acc, ray_drop = render_rays(model, rays_o, rays_d, near=1., far=64., N_samples=N_samples)
    end = time.time()
    new_point_cloud_spherical = np.zeros([np.shape(depth)[0]*np.shape(depth)[1],3])
    
    depth = tf.transpose(depth).numpy() #need this
    depth = np.flip(depth, axis = 0) #needed

    ray_drop = tf.transpose(ray_drop).numpy() #test
    ray_drop = np.flip(ray_drop, axis = 0) #test
    
    phimin = -0.53529248 #rad
    phimax = 0.18622663 #rad

    count = 0
    for w in range(np.shape(pcs)[1]):
        for h in range(np.shape(pcs)[0]):
            #draw all points
#             new_point_cloud_spherical[count,0] = depth[w,h] #radius
            #suppress ray dropped points
            if ray_drop[w,h] > 0.9:
                    new_point_cloud_spherical[count,0] = depth[w,h] #radius
            else:
                    new_point_cloud_spherical[count,0] = 0#100

            new_point_cloud_spherical[count,1] = (phimax-phimin)*(w/(np.shape(depth)[0])) #theta (square)
#             new_point_cloud_spherical[count,1] = (phimax-phimin)*(w/(np.shape(depth)[0]))/8 #theta (eighth)
#             new_point_cloud_spherical[count,1] = (phimax-phimin)*(w/(np.shape(depth)[0]))/16 #theta (1/16)
#             new_point_cloud_spherical[count,1] = (phimax-phimin)*(w/(np.shape(depth)[0]))/32 #theta (1/32)
            new_point_cloud_spherical[count,2] = np.pi/2 + phimax - (phimax-phimin)*(h/np.shape(depth)[1]) #phi
            count+= 1

    #account for sweep angle using j
    new_point_cloud_spherical[:,1] -= j*(phimax-phimin) #square
#     new_point_cloud_spherical[:,1] -= j*(phimax-phimin)/8 #eighth
#     new_point_cloud_spherical[:,1] -= j*(phimax-phimin)/16 #1/16
#     new_point_cloud_spherical[:,1] -= j*(phimax-phimin)/32 #1/32

    #if last square patch in sweep, throw away any points that wrap around origin
    if j==8:
        new_point_cloud_spherical= new_point_cloud_spherical[new_point_cloud_spherical[:,1]>=(-2*np.pi + (phimax-phimin))]
    
    new_point_cloud_cart = LC.s2c(LC,new_point_cloud_spherical).numpy()
    new_point_cloud_cart[:,2] = -new_point_cloud_cart[:,2] #need to flip z 

    # rainbow by z height
    zheight = 100*(np.sin(0.25*new_point_cloud_cart[:,2])+1)
    cname = np.array([1-zheight, zheight, 1.5*zheight]).T.tolist()
    disp.append(Points(new_point_cloud_cart, c = cname, r = 3, alpha = 0.5))
    
#     pcfn = "gazebo_scene2/box2_scan" + str(3) + ".npy" #box v3
#     pc = np.load(pcfn)
#     disp.append(Points(pc, c = 'k', r = 3, alpha = 0.3))
    
plt.show(disp, "Novel Point Cloud From NeRF")
ViewInteractiveWidget(plt.window)

# Animate GIF of sensor moving through scene

In [ ]:
num_rots = 9
num_frames = 50

for i in range(num_frames):
    print(i)
    plt = Plotter(N = 1, axes = 4, bg = (1, 1, 1), offscreen=True)
    disp=[] 
    for j in range(num_rots):
        #get sensor transformation matrix
        rotm = np.eye(4)

        # account for image crop in rotation -------------------
        crop_angle = -(phimax-phimin)/2 - j*(phimax-phimin)#square
#         crop_angle = -(phimax-phimin)/32 - j*(phimax-phimin)/32 #1/32
        rotm_crop = R.from_euler('xyz', [0,-crop_angle,0]).as_matrix()
        rotm[:3,:3] = rotm[:3,:3] @ rotm_crop

        #also need to account for the fact that the LIDAR beam isn't actually centered at horizon
        sensor_elevation_zero_rotm = R.from_euler('xyz', [(phimin+phimax)/2,0,0]).as_matrix() #test
        rotm[:3,:3] = rotm[:3,:3] @ sensor_elevation_zero_rotm
        # ------------------------------------------------------

        # flip x and z axis
        rotm[0,-1], rotm[2,-1] = rotm[2,-1], rotm[0,-1] 
        rotm[0:3,2] *= -1 # flip sign of y and z axis
        rotm[0:3,1] *= -1
        rotm = rotm[[1,0,2,3],:]
        rotm[2,:] *= -1 # flip whole world upside down
        rotm[2,-1] += 30 - i/2 #x in world frame output
        rotm[0,-1] = -2 #- (i/3) #z in world frame output
        rotm[1,-1] = 2 # + (i/6) #y in world frame
        rotm = rotm.astype(np.float32)

        #call NeRF using specified novel rotm
        rays_o, rays_d = get_rays(H, W, focal, rotm)
        depth, acc, ray_drop = render_rays(model, rays_o, rays_d, near=1., far=64., N_samples=N_samples)
        end = time.time()
        new_point_cloud_spherical = np.zeros([np.shape(depth)[0]*np.shape(depth)[1],3])

        #need this
        depth = tf.transpose(depth).numpy() 
        depth = np.flip(depth, axis = 0) 
        ray_drop = tf.transpose(ray_drop).numpy()
        ray_drop = np.flip(ray_drop, axis = 0)
        phimin = -0.53529248 #rad
        phimax = 0.18622663 #rad

        count = 0
        for w in range(np.shape(pcs)[1]):
            for h in range(np.shape(pcs)[0]):
                #draw all points
    #             new_point_cloud_spherical[count,0] = depth[w,h] #radius
                #suppress ray dropped points
                if ray_drop[w,h] > 0.9:
                        new_point_cloud_spherical[count,0] = depth[w,h] #radius
                else:
                        new_point_cloud_spherical[count,0] = 0#100

                
                new_point_cloud_spherical[count,1] = (phimax-phimin)*(w/(np.shape(depth)[0])) #theta (square)
#                 new_point_cloud_spherical[count,1] = (phimax-phimin)*(w/(np.shape(depth)[0]))/32 #theta (1/32)
                new_point_cloud_spherical[count,2] = np.pi/2 + phimax - (phimax-phimin)*(h/np.shape(depth)[1]) #phi
                count+= 1

        #account for sweep angle using j
        new_point_cloud_spherical[:,1] -= j*(phimax-phimin) #square
#         new_point_cloud_spherical[:,1] -= j*(phimax-phimin)/32 #1/32            

        #if last square patch in sweep, throw away any points that wrap around origin
        if j==8:
            new_point_cloud_spherical= new_point_cloud_spherical[new_point_cloud_spherical[:,1]>=(-2*np.pi + (phimax-phimin))]

        new_point_cloud_cart = LC.s2c(LC,new_point_cloud_spherical).numpy()
        new_point_cloud_cart[:,2] = -new_point_cloud_cart[:,2] #need to flip z 

        #translate to keep camera fixed in place
        new_point_cloud_cart[:,0] += np.cos(-(phimax-phimin)) * (i/2)
        new_point_cloud_cart[:,1] -= np.sin(-(phimax-phimin)) * (i/2)
        
        # rainbow by z height
        zheight = 100*(np.sin(0.25*new_point_cloud_cart[:,2])+1)
        cname = np.array([1-zheight, zheight, 1.5*zheight]).T.tolist()
        disp.append(Points(new_point_cloud_cart, c = cname, r = 3., alpha = 0.5))
    
    #overhead diagonal
    cam = dict(
        pos=(-23.00344, -94.74697, 90.65512),
        focalPoint=(-3.497593, 3.745954, 0.06619059),
        viewup=(0.1094000, 0.6610702, 0.7423057),
        distance=135.2320,
        clippingRange=(44.94560, 235.8997),
    )
#     #follow cam
#     cam = dict(
#         pos=(-43.92513, -38.84776, 18.14660),
#         focalPoint=(-2.117923, 1.000159, 2.273724),
#         viewup=(0.2240760, 0.1485759, 0.9631797),
#         distance=59.89697,
#         clippingRange=(0.2096860, 209.6860),
#     )

    plt.show(disp, "Novel Point Cloud From NeRF at [" 
             + str(-rotm[2,-1]) + ", " + str(-rotm[1,-1]) + ", " + str(-rotm[0,-1]) + "]",
             camera= cam).screenshot("lidar_nerf_demo/town1_" + str(i) + ".png")
    plt.clear()
    plt.close()
# ViewInteractiveWidget(plt.window)

## Loss Function:
# $\mathcal{L}_{total} = \mathcal{L}_{distance} + \lambda_{1}\mathcal{L}_{intensity}(r) + \lambda_{2}\mathcal{L}_{raydrop}(r) + \lambda_{3}\mathcal{L}_{reg}(r)$

## $\mathcal{L}_{distance}(r) = \Sigma_{r \in R} || \hat{D}(r) - D(r)||$

R is the set of all rays, $\lambda$ are the weights for each term

## $\mathcal{L}_{raydrop} = \frac{1}{|R|} \sum_{r\in\mathbb{R}} 
(\mathcal{L}_{bce} + \mathcal{L}_{ls}) $

$\mathcal{L}_{bce}$ is binary cross entropy. 

$\mathcal{L}_{ls}$ is Lovasz loss, a commonly used metric for optimizing intersection-over-union

$\lambda_1 = 1, \lambda_2 = 1, \lambda_3 = 1e2$

In [ ]:
true_mask = testimg[:,:,1:]
# true_mask = images[42,:,:,1:]

fig, ax = p.subplots(1,2)
ax[0].imshow(true_mask,cmap = "gray")
ax[0].set_title("true depth mask")

# dummy_mask = testimg[:,:,1:]
# #add false positives
dummy_mask = (testimg[:,:,1:] * np.floor(0.99 + np.random.rand(np.shape(testimg)[0], np.shape(testimg)[1]))[:,:,None])
#add slight underscaling
dummy_mask = dummy_mask * (9 + np.random.rand(np.shape(testimg)[0], np.shape(testimg)[1])[:,:,None])/10
# print(dummy_mask[:,:,0])
ax[1].imshow(dummy_mask, cmap = "gray")
# ax[1].imshow(ray_drop, cmap = "gray")
ax[1].set_title("ray drop esimate")

print(np.shape(true_mask))

loss = tf.keras.losses.binary_crossentropy(true_mask, dummy_mask)
# print(loss)
print(tf.reduce_sum(loss))

# Debug: visualize frames for drawing points for depth testing

In [ ]:
#GOAL: try remaking <dirs> but with spherical projection (not pinhole camera intrinsics)
from scipy.spatial.transform import Rotation as R

i, j = tf.meshgrid(tf.range(W, dtype=tf.float32), tf.range(H, dtype=tf.float32), indexing='xy')
look_at_pose = 5
c2w = poses[look_at_pose]
# c2w = np.eye(3)
c2w = tf.cast(c2w, tf.float32)
near = 0.
far = 64.
focal = np.array(np.shape(images)[1]/(2*np.tan((phimax-phimin)/2))) #default image size

plt = Plotter(N = 2, axes = 1, bg = (1, 1, 1), interactive = True, sharecam = False) #axes = 4 (simple), 1(scale)
disp=[]
disp_alt = []

#pinhole camera projection ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
dirs = tf.stack([(i-(W*.5))/focal, -(j-(H*.5))/focal, -tf.ones_like(i)], -1) 
# c2w = np.eye(3)
rays_d = tf.reduce_sum(dirs[..., np.newaxis, :] * c2w[:3,:3], -1)
rays_o = tf.broadcast_to(c2w[:3,-1], tf.shape(rays_d))
z_vals = tf.linspace(near, far, N_samples) 
z_vals += tf.random.uniform(list(rays_o.shape[:-1]) + [N_samples]) * (far-near)/N_samples
#[image_height, image_width, batch_size, 3]
pts = rays_o[...,None,:] + rays_d[...,None,:] * z_vals[...,:,None]
pts_flat = tf.reshape(pts, [-1,3])

disp.append(Points(tf.reshape(rays_d, [-1,3]), c = 'light blue',r=5, alpha = 0.5)) #compare here, dirs doesn't invoke c2w
disp.append(Points(tf.reshape(rays_d, [-1,3])[:1000], c = 'blue',r=10, alpha = 0.5))
disp_alt.append(Points(pts_flat, c = 'blue', r = 5, alpha = 0.01))
disp_alt.append(Points(pts_flat[:100], c = 'blue', r = 10, alpha = 0.2))
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

#reformat for LiDAR depth measurements ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
vert_fov = np.rad2deg(phimax-phimin)
# #"square" lidar image
# dirs_test = tf.stack([-tf.ones_like(i), np.pi/2+(j-(W*.5))/focal, (i-(H*.5))/focal], -1) 
# non-horizon centered lidar image
dirs_test = tf.stack([-tf.ones_like(i), -(j-(W*.5))/(2*vert_fov) + np.rad2deg(phimax), (i-(H*.5))/(2*vert_fov)], -1) #looks right (but does not converge perfectly)
# dirs_test = tf.stack([-tf.ones_like(i), -(j-(W*.5))/focal - np.rad2deg(phimax-phimin)/2, (i-(H*.5))/focal], -1) #test

dirs_test = tf.reshape(dirs_test,[-1,3])
#[r, theta, phi] --> [r, phi, theta]
dirs_test = LC.s2c(LC, tf.transpose(tf.Variable([dirs_test[:,0], dirs_test[:,2], dirs_test[:,1]], dtype = tf.float32)))

# #test-- try projecting skewed depth image flat
# dirs_test = tf.transpose(tf.Variable([tf.ones(len(dirs_test)), dirs_test[:,1], dirs_test[:,2]], dtype = tf.float32))

disp.append(Points(dirs_test, c = 'light green'))
disp.append(Points(dirs_test[:1000], r = 10, c = 'green'))

#need to rotate red points into same frame as blue points 
# rotm = R.from_euler('xyz', [0,np.pi/2,0]).as_matrix() #square
# rotm = R.from_euler('xyz', [0,-np.rad2deg(phimax-phimin)/2,0]).as_matrix() #non-horizon centered lidar image v1
# rotm = R.from_euler('xyz', [0,np.rad2deg(phimax-phimin)/2,0]).as_matrix() #need to flip sign for upside down test
#try out accounting for horizon misalignment here
# rotm = R.from_euler('xyz', [0,np.rad2deg(phimax),0]).as_matrix()
rotm = R.from_euler('xyz', [0,-np.pi/2,0]).as_matrix() #test
# rotm = R.from_euler('xyz', [0,-(np.pi/2)+phimax,0]).as_matrix() #test


dirs_test = dirs_test @ rotm
dirs_test = dirs_test @ tf.transpose(c2w[:3,:3]) #need this to get dirs to align (but not for pts_flat)

# aligns dirs, doesn't get pts_flat to work yet
dirs_test = dirs_test @ (c2w[:3,:3] 
                      @ R.from_euler('xyz', [0,0,np.pi/2]).as_matrix() #looked good but converged slightly off
#                       @ R.from_euler('xyz', [0,0,np.pi/2]).as_matrix()  #test
                      @ np.linalg.pinv(c2w[:3,:3]) )

# dirs_test = dirs_test @ (c2w[:3,:3] 
#                       @ R.from_euler('xyz', [0,np.pi,0]).as_matrix() 
#                       @ np.linalg.pinv(c2w[:3,:3]) )

# rays_d_test = tf.reduce_sum(dirs_test[..., np.newaxis, :] * c2w[:3,:3], -1) #supposed to be this
rotm_fix = (c2w[:3,:3] 
#             @ R.from_euler('xyz', [0,0,-np.pi/2]).as_matrix() 
#             @ R.from_euler('xyz', [0,0,np.pi/2]).as_matrix() 
            @ np.linalg.pinv(c2w[:3,:3]))
#             @ c2w[:3,:3] 
#             @ R.from_euler('xyz', [0,np.pi,0]).as_matrix() 
#             @ np.linalg.pinv(c2w[:3,:3]))
rays_d_test = tf.reduce_sum(dirs_test[..., np.newaxis, :] * rotm_fix, -1) #looks like I need to do this instead
rays_o_test = tf.broadcast_to(c2w[:3,-1], tf.shape(rays_d_test))
z_vals_test = tf.linspace(near, far, N_samples) 
z_vals_test += tf.random.uniform(list(rays_o_test.shape[:-1]) + [N_samples]) * (far-near)/N_samples
#[image_height, image_width, batch_size, 3]
pts_test = rays_o_test[...,None,:] + rays_d_test[...,None,:] * z_vals_test[...,:,None]
pts_flat_test = tf.reshape(pts_test, [-1,3])

disp.append(Points(dirs_test, c = 'pink', r = 5, alpha = 0.5))
disp.append(Points(dirs_test[:1000], c = 'red', r = 10, alpha = 0.5))
disp_alt.append(Points(pts_flat_test, c = 'red', r = 5, alpha = 0.01))
disp_alt.append(Points(pts_flat_test[:100], c = 'red', r = 10, alpha = 0.5))
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

#draw frames
#forward view direction (-z in NeRF c2w convention)
headings = poses[look_at_pose,:3,:3] @ np.array([0,0,-1])
disp_alt.append(Arrows(poses[look_at_pose,:3,-1][None,:], (poses[look_at_pose,:3,-1] + headings)[None,:], c = "yellow"))
disp.append(Arrows(np.zeros([3,1]), (np.zeros([3,1]) + headings), c = "yellow"))
# x
headings = poses[look_at_pose,:3,:3] @ np.array([1,0,0])
disp_alt.append(Arrows(poses[look_at_pose,:3,-1][None,:], (poses[look_at_pose,:3,-1] + headings)[None,:], c = "red"))
disp.append(Arrows(np.zeros([3,1]), (np.zeros([3,1]) + headings), c = "red"))
#y
headings = poses[look_at_pose,:3,:3] @ np.array([0,1,0])
disp_alt.append(Arrows(poses[look_at_pose,:3,-1][None,:], (poses[look_at_pose,:3,-1] + headings)[None,:], c = "green"))
disp.append(Arrows(np.zeros([3,1]), (np.zeros([3,1]) + headings), c = "green"))
#z
headings = poses[look_at_pose,:3,:3] @ np.array([0,0,1])
disp_alt.append(Arrows(poses[look_at_pose,:3,-1][None,:], (poses[look_at_pose,:3,-1] + headings)[None,:], c = "blue"))
disp.append(Arrows(np.zeros([3,1]), (np.zeros([3,1]) + headings), c = "blue"))


plt.show(disp, "dirs", at = 0)
plt.show(disp_alt, "pts_flat", at = 1)
ViewInteractiveWidget(plt.window)

In [ ]:
# # focal = np.array(np.shape(images)[1]/(2*np.tan((phimax-phimin)/2))) #default image size
# print(focal) #in pixels
# # vert_fov = np.rad2deg(phimax-phimin)
# print(vert_fov) #in degrees
# print(np.rad2deg(phimax-phimin)/2)
# print(phimax, phimin)

print((phimin-phimax)/2)
print(np.tan((phimin-phimax)/2))
print(np.arctan((phimin-phimax)/2))

In [ ]:
#debug -- draw multiple poses old camera projection and new lidar projection on same graph
#         -> Look for areas of overlap that may be poisoning training

num_patches = 3

phimin = -0.53529248 #rad
phimax = 0.18622663 #rad

#test
# phimin, phimax = -(phimax-phimin)/2, (phimax-phimin)/2

plt = Plotter(N = 1, axes = 0, bg = (1, 1, 1), interactive = True, sharecam = False) #axes = 4 (simple), 1(scale)
disp=[]

for count in range(num_patches):

    i, j = tf.meshgrid(tf.range(W, dtype=tf.float32), tf.range(H, dtype=tf.float32), indexing='xy')
    look_at_pose = count
    c2w = poses[look_at_pose]
    # c2w = np.eye(3)
    c2w = tf.cast(c2w, tf.float32)
    near = 0.
    far = 64.
    focal = np.array(np.shape(images)[1]/(2*np.tan((phimax-phimin)/2))) #default image size

    #pinhole camera projection ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    dirs = tf.stack([(i-(W*.5))/focal, -(j-(H*.5))/focal, -tf.ones_like(i)], -1) 
    # c2w = np.eye(3)
    rays_d = tf.reduce_sum(dirs[..., np.newaxis, :] * c2w[:3,:3], -1)
    rays_o = tf.broadcast_to(c2w[:3,-1], tf.shape(rays_d))
    z_vals = tf.linspace(near, far, N_samples) 
    z_vals += tf.random.uniform(list(rays_o.shape[:-1]) + [N_samples]) * (far-near)/N_samples

    disp.append(Points(tf.reshape(rays_d, [-1,3]), c = 'light blue',r=5, alpha = 0.75)) #compare here, dirs doesn't invoke c2w
    # disp.append(Points(tf.reshape(rays_d, [-1,3])[:1000], c = 'blue',r=10, alpha = 0.5))
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

    #reformat for LiDAR depth measurements ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    vert_fov = np.rad2deg(phimax-phimin)
    # #"square" lidar image
    # dirs_test = tf.stack([-tf.ones_like(i), np.pi/2+(j-(W*.5))/focal, (i-(H*.5))/focal], -1) 
    # non-horizon centered lidar image
#     dirs_test = tf.stack([-tf.ones_like(i), -(j-(W*.5))/(2*vert_fov) + np.rad2deg(phimax), (i-(H*.5))/(2*vert_fov)], -1) #looks right (but does not converge perfectly)
    #looks like there is an issue with horizontal overlap
#     dirs_test = tf.stack([-tf.ones_like(i), -(j-(W*.5))/(2*vert_fov) - (np.pi/2) - phimax, (i-(H*.5))/(2*vert_fov)], -1)
    dirs_test = tf.stack([-tf.ones_like(i), 
                          -(j-(W*.5))/(focal) - (np.pi/2) - phimax, 
                          np.arctan((i-(H*.5))/(focal))], -1)
    
    dirs_test = tf.reshape(dirs_test,[-1,3])
    #[r, theta, phi] --> [r, phi, theta]
    dirs_test = LC.s2c(LC, tf.transpose(tf.Variable([dirs_test[:,0], dirs_test[:,2], dirs_test[:,1]], dtype = tf.float32)))

    # #test-- try projecting skewed depth image flat
    # dirs_test = tf.transpose(tf.Variable([tf.ones(len(dirs_test)), dirs_test[:,1], dirs_test[:,2]], dtype = tf.float32))

    # disp.append(Points(dirs_test, c = 'light green'))
    # # disp.append(Points(dirs_test[:1000], r = 10, c = 'green'))

    #need to rotate red points into same frame as blue points 
    # rotm = R.from_euler('xyz', [0,np.pi/2,0]).as_matrix() #square
    # rotm = R.from_euler('xyz', [0,np.rad2deg(phimax-phimin)/2,0]).as_matrix() #need to flip sign for upside down test
    #try out accounting for horizon misalignment here
    # rotm = R.from_euler('xyz', [0,np.rad2deg(phimax),0]).as_matrix()
#     rotm = R.from_euler('xyz', [0,-np.pi/2,0]).as_matrix() #test
#     rotm = R.from_euler('xyz', [0,-(np.pi/2)-(phimax),0]).as_matrix() #better
    rotm = R.from_euler('xyz', [0,-(np.pi/2)+(((phimax+phimin))/2),0]).as_matrix() #test

    dirs_test = dirs_test @ rotm
    dirs_test = dirs_test @ tf.transpose(c2w[:3,:3]) #need this to get dirs to align (but not for pts_flat)

    # aligns dirs, doesn't get pts_flat to work yet
    dirs_test = dirs_test @ (c2w[:3,:3] 
                          @ R.from_euler('xyz', [0,0,np.pi/2]).as_matrix() #looked good but converged slightly off
    #                       @ R.from_euler('xyz', [0,0,np.pi/2]).as_matrix()  #test
                          @ np.linalg.pinv(c2w[:3,:3]) )

    # dirs_test = dirs_test @ (c2w[:3,:3] 
    #                       @ R.from_euler('xyz', [0,np.pi,0]).as_matrix() 
    #                       @ np.linalg.pinv(c2w[:3,:3]) )

    # rays_d_test = tf.reduce_sum(dirs_test[..., np.newaxis, :] * c2w[:3,:3], -1) #supposed to be this
    rotm_fix = (c2w[:3,:3] 
    #             @ R.from_euler('xyz', [0,0,-np.pi/2]).as_matrix() 
    #             @ R.from_euler('xyz', [0,0,np.pi/2]).as_matrix() 
                @ np.linalg.pinv(c2w[:3,:3]))
    #             @ c2w[:3,:3] 
    #             @ R.from_euler('xyz', [0,np.pi,0]).as_matrix() 
    #             @ np.linalg.pinv(c2w[:3,:3]))
    rays_d_test = tf.reduce_sum(dirs_test[..., np.newaxis, :] * rotm_fix, -1) #looks like I need to do this instead
    rays_o_test = tf.broadcast_to(c2w[:3,-1], tf.shape(rays_d_test))
    z_vals_test = tf.linspace(near, far, N_samples) 
    z_vals_test += tf.random.uniform(list(rays_o_test.shape[:-1]) + [N_samples]) * (far-near)/N_samples
    #[image_height, image_width, batch_size, 3]
    pts_test = rays_o_test[...,None,:] + rays_d_test[...,None,:] * z_vals_test[...,:,None]
    pts_flat_test = tf.reshape(pts_test, [-1,3])

#     cname = np.array([255*(np.random.rand()), 255*(np.random.rand()), 255*(np.random.rand())]).T.tolist()
#     disp.append(Points(dirs_test, c = cname, r = 5, alpha = 0.5))
    disp.append(Points(dirs_test, c = 'pink', r = 5, alpha = 0.35))
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

    #draw frames first pose
    #forward view direction (-z in NeRF c2w convention) 
    headings = poses[look_at_pose,:3,:3] @ np.array([0,0,-0.3])
    disp.append(Arrows(np.zeros([3,1]), (np.zeros([3,1]) + headings), c = "yellow"))
    # x
    headings = poses[look_at_pose,:3,:3] @ np.array([0.3,0,0])
    disp.append(Arrows(np.zeros([3,1]), (np.zeros([3,1]) + headings), c = "red"))
    #y
    headings = poses[look_at_pose,:3,:3] @ np.array([0,0.3,0])
    disp.append(Arrows(np.zeros([3,1]), (np.zeros([3,1]) + headings), c = "green"))
    #z
    headings = poses[look_at_pose,:3,:3] @ np.array([0,0,0.3])
    disp.append(Arrows(np.zeros([3,1]), (np.zeros([3,1]) + headings), c = "blue"))


plt.show(disp, "dirs", at = 0)
ViewInteractiveWidget(plt.window)

In [ ]:
# model.save_weights("models/townV1.ckpt")
# model.load_weights("models/townV1.ckpt")
# # print(model.summary())

# #test for visualization with Netron
# model.save('models/townV1.keras')

In [ ]:
# tf.keras.utils.plot_model(
#     model,
#     to_file='model.png',
#     show_shapes=False,
#     show_dtype=False,
#     show_layer_names=False,
#     rankdir='TB',
#     expand_nested=False,
#     dpi=200,
#     show_layer_activations=False,
#     show_trainable=False,
#     **kwargs
# )
# tf.keras.utils.plot_model(model)
dim = (64,64,1)
model.build((None, *dim))
model.build_graph().summary()

In [ ]:
# i, j = tf.meshgrid(tf.range(W, dtype=tf.float32), tf.range(H, dtype=tf.float32), indexing='xy')
# #pinhole camera projection
# # dirs = tf.stack([(i-W*.5)/focal, -(j-H*.5)/focal, -tf.ones_like(i)], -1) 
# #test-- reformat for LiDAR depth measurements
# vert_fov = np.rad2deg(phimax-phimin)
# # print(focal)
# print(vert_fov)
# dirs = tf.stack([(i-(W*.5))/focal, -(j-(H*.5))/focal, -tf.ones_like(i)], -1) #pinhole camera projection

# # print(np.shape(dirs))

# # c2w = poses[0]
# c2w = np.eye(3)
# c2w = tf.cast(c2w, tf.float32)
# rays_d = tf.reduce_sum(dirs[..., np.newaxis, :] * c2w[:3,:3], -1)
# rays_o = tf.broadcast_to(c2w[:3,-1], tf.shape(rays_d))
# # print(N_samples)

# near = 0.
# far = 64.
# z_vals = tf.linspace(near, far, N_samples) 
# z_vals += tf.random.uniform(list(rays_o.shape[:-1]) + [N_samples]) * (far-near)/N_samples
# #[image_height, image_width, batch_size, 3]
# pts = rays_o[...,None,:] + rays_d[...,None,:] * z_vals[...,:,None]
# pts_flat = tf.reshape(pts, [-1,3])

In [ ]:
# 3+3*(2**10)
3*(2**4)

In [ ]:
# 63 params in positional branch
3+3*2**4

In [ ]:
print(np.rad2deg(phimax-phimin))